# Optimized Matrix Multiplication Validation

In [3]:
import numpy as np
np.set_printoptions(linewidth=400)
from fxpmath import Fxp

## Generate Index Array from Matrix

In [4]:
def check_one_hot(matrix):
    return np.all(np.sum(matrix, axis=1) == 1)

In [26]:
# process input matrix into array which stores necessary positions
# format of array is: [left#, right#, target#, 'operation']
# when target# is equal with -1, the values store in left# and right# will be the output value
def preprocess_matrix_to_array(matrix):
    # print matrix
    print("Size：",matrix.size)      
    print("Shape：",matrix.shape)        
    print("Dimension",matrix.ndim)      
    print(matrix,"\n")
    
    # define parameters
    rows1, cols1 = matrix.shape
    bit = int(np.ceil(np.log2(np.max(np.abs(matrix))+1)))+1
    bin_weight = cols1*bit
    
    # the first row of index_array is to store required parameters, [rows, cols, bit, 'parameters']
    index_array = []
    index_array.append([rows1, cols1, bit,'parameters'])
    
    
    # prepocess input matrix into binary matrix
    bin_matrix = np.zeros(shape=(rows1, bin_weight),dtype=np.int32)
    for i in range(0, rows1):
        for j in range(0, cols1):
            a = np.binary_repr(matrix[i][j],bit)
            for k in range(0,bit):
                    bin_matrix [i][k+bit*j]=a[k]
     
    a = 1
    for i in range(bin_weight):
        if((i%bit)==0):
            print("MEM[%d] = -(input_vector[%d] << %d);" %(i,a-1,a*bit-i-1))
            index_array.append([i,a-1,a*bit-i-1,'-<<'])
        else:
            print("MEM[%d] = input_vector[%d] << %d;" %(i,a-1,a*bit-i-1))
            index_array.append([i,a-1,a*bit-i-1,'<<'])
        if((i+1)%bit==0):
            a+=1
    
    rows2, cols2 = bin_matrix.shape
    # find the number of max matching ones of input binary matrix
    max_matching = 0
    for i in range(cols2 - 1):
        for j in range(i + 1, cols2):
            matching = np.sum((bin_matrix[:, i] == 1) & (bin_matrix[:, j] == 1))
            if matching > max_matching:
                max_matching = matching
                
    # process binary matrix
    cnt = 0 #count number of matches
    add_count = [0]*rows2 #count number of additions 
    one_count = max_matching #count number of one in column
    loop_range = cols2
    while(one_count):
        print("\n Max number of 1: %d" %(one_count))
        for i in range(0,loop_range):
            for j in range(i+1,loop_range):
                if((np.sum(bin_matrix[:,i]==1))>=one_count):
                    if((np.count_nonzero(bin_matrix[:,[i]]&bin_matrix[:,[j]]))==one_count): 
                        loop_range+=1
                        print("MEM[%d] = MEM[%d] + MEM[%d];"%(bin_weight+cnt,i,j))
                        index_array.append([i,j,bin_weight+cnt,'+'])
                        bin_matrix = np.hstack((bin_matrix, np.zeros((rows1, 1), dtype=np.int32)))
                        bin_matrix[:,[bin_weight+cnt]] = bin_matrix[:,[i]]&bin_matrix[:,[j]]
                        bin_matrix[:,[i]] = bin_matrix[:,[i]]&(~bin_matrix[:,[bin_weight+cnt]])
                        bin_matrix[:,[j]] = bin_matrix[:,[j]]&(~bin_matrix[:,[bin_weight+cnt]])
                        cnt+=1
                        #new_cols_added = True
                        add_count[rows2-one_count] += 1
        print("Counts of Additions for %d one match:  %d" %(one_count,add_count[rows2-one_count]))
        if(one_count>1):
            one_count=one_count-1
        else:
            one_count = 1
        if check_one_hot(bin_matrix):
            break
    print("Total Counts of Additions:%d \n" %(cnt))
    
    # find output index
    output_index = []
    # Iterate over the rows
    for row_index, row in enumerate(bin_matrix):
        # Iterate over the columns
        for col_index, value in enumerate(row):
            if value == 1:
                output_index.append((row_index,col_index))
                break  
    h = 0
    for i in range(rows2):
        print("output_vector[%d] = MEM[%d]; " %(output_index[h][0], output_index[h][1]))
        index_array.append([h,output_index[h][1],-1,'='])
        h += 1
    
    # print binary matrix
    print("\nSize：",bin_matrix .size)      
    print("Shape：",bin_matrix .shape)         
    print("Dimension",bin_matrix .ndim)      
    print(bin_matrix, "\n")
    
    # print index array 
    for row in index_array:
        print(row)
        
    return index_array

In [6]:
def minimize_operation_matrix(matrix, matrix_binarized = False, bit_size = 8):
    """
    Args:
        matrix: given m x n matrix
        bit_size: bit width of each element in the matrix
        matrix_binarized: flag to show whether matrix is already binarized or not
        
    Output:
        index_array: format of array is: [left#, right#, target#, 'operation'] 
            and first row is: [rows, cols, bit, 'parameters']
    """
    # bit_size = bit_size+1
    rows1, cols1 = matrix.shape 
    bin_max_length = cols1*bit_size*3
    bin_weight = cols1*bit_size
    
    if not matrix_binarized:
        bin_matrix_ext = np.zeros( (rows1, bin_max_length), dtype=np.uint8)
        # Element wise conversion of each element to binary number for 2^bit notation
        bin_matrix = ((matrix.reshape(-1, 1) & (2**np.arange(bit_size)[::-1])) != 0).astype(int)
        bin_matrix = bin_matrix.reshape(rows1, -1)
        # Copying elements of bin matrix to extended binarized matrix
        bin_matrix_ext[:,:bit_size*cols1] = bin_matrix
    else:
        bin_matrix_ext = matrix
    
    # find the number of max ones of input binary matrix
    max_matching = bin_matrix_ext.sum(axis=0).max()
  
    # process binary matrix
    rows2, cols2 = bin_matrix_ext.shape
    cnt = 0 #count number of matches
    add_count = [0]*rows2 #count number of additions 
    one_count = int(max_matching) #count number of one in column

    # the first row of index_array is to store required parameters, [rows, cols, bit_size, 'parameters']
    index_array = []
    index_array.append([rows1, cols1, bit_size,'parameters'])

    a = 1
    for i in range(bin_weight):
        if((i%bit_size)==0):
            #print("MEM[%d] = -(input_vector[%d] << %d);" %(i,a-1,a*bit_size-i-1))
            index_array.append([i,a-1,a*bit_size-i-1,'-<<'])
        else:
            #print("MEM[%d] = input_vector[%d] << %d;" %(i,a-1,a*bit_size-i-1))
            index_array.append([i,a-1,a*bit_size-i-1,'<<'])
        if((i+1)%bit_size==0):
            a+=1
    
    
    while(one_count >= 1):
        print("\n Max number of 1: %d" % one_count)
        i = 0
        # while(i < cols2 and bin_max_length > bin_weight+cnt):
        while(i < bin_matrix_ext.shape[1]): # and bin_max_length > bin_weight+cnt):
            # doubling the matrix size whenever it runs out of memory  
            if bin_max_length <= bin_weight+cnt+2: 
                print('Increasing matrix size, current bin_max_length:', bin_max_length, 'bin_weight:', bin_weight, 'cnt:', cnt)
                bin_max_length += bin_matrix_ext.shape[1]
                bin_matrix_ext = np.hstack((bin_matrix_ext, np.zeros((rows1, bin_matrix_ext.shape[1]), 
                                                            dtype=np.int32)))
                print('Increasing matrix size, updated bin_max_length:', bin_max_length)
            
            # print('i:', i, 'searching for col:', bin_matrix_ext[:,i])
            # print('one_count:', one_count, 'i:', i)
            if((np.sum(bin_matrix_ext[:,i]==1))>=one_count):
                matching_idx = ((bin_matrix_ext[:,i] @ bin_matrix_ext) == one_count).nonzero()[0]
                # print('matching_idx:', matching_idx)
                if len(matching_idx) > 1:
                    j = matching_idx[0] if i != matching_idx[0] else matching_idx[1]
                    # print('j:', j, 'found col:', bin_matrix_ext[:,j])
                    #print("MEM[%d] = MEM[%d] + MEM[%d];"%(bin_weight+cnt,i,j))
                    index_array.append([i,j,bin_weight+cnt,'+'])
                    bin_matrix_ext[:,bin_weight+cnt] = bin_matrix_ext[:,i]&bin_matrix_ext[:,j]
                    bin_matrix_ext[:,i] = bin_matrix_ext[:,i]&(~bin_matrix_ext[:,bin_weight+cnt])
                    bin_matrix_ext[:,j] = bin_matrix_ext[:,j]&(~bin_matrix_ext[:,bin_weight+cnt])

                    cnt += 1
                    add_count[rows2-one_count] += 1
                else:
                    i += 1
            else:
                i += 1
        
        if check_one_hot(bin_matrix_ext):
            break
        #print("Counts of Additions for %d one match:  %d" %(one_count,add_count[rows2-one_count]))
        one_count=one_count-1
    
    # Adding non zero operations at the end
    nonzero_row_cols = bin_matrix_ext.nonzero() 
    nonzero_rows = nonzero_row_cols[0]
    nonzero_cols = nonzero_row_cols[1]
    index_array = index_array + [[row, col, -1, '='] for row, col in zip(nonzero_rows, nonzero_cols)]
    
    return index_array, bin_matrix_ext

## Validate Multiplication Results between Index Array and Input vector

In [27]:
# matrix optimized multiplication between matrix and input_vector
def matrix_optimized_multiplier(index_array, input_vector):
    """
    Args:
        index_array: format of array is: [left#, right#, target#, 'operation'] 
            and first row is: [rows, cols, bit, 'parameters']
        input_vector: given n sized input
        
    Output:
        output_vector: resulting matrix multiplication of input_vector and 
            the index array reduced matrix. 
    """
    # define parameters
    rows1, cols1 = index_array[0][0], index_array[0][1]
    bit = index_array[0][2]    
    # mem_size = max(row[2] for row in index_array)
    mem_size = len(index_array)
    
    #initialize output vector
    output_vector = np.zeros(shape=(rows1, 1),dtype=np.int32)
    
    #initialize MEM
    MEM = np.zeros(shape=(mem_size+1),dtype=np.int32)
    
    # do calculations 
    # row format: [left_pos, right_pos, target#, 'operation'] 
    for row in index_array[1:]:
        # print(row)
        if row[2] != -1:
            if row[3] == '-<<':
                MEM[row[0]] = -1 * pow(2,row[2]) * input_vector[row[1]]
            #here actually we don't need to do >> operation.
            elif row[3] == '>>':
                MEM[row[0]] = pow(2,-row[2]) * input_vector[row[1]]
            elif row[3] == '<<':
                MEM[row[0]] = pow(2,row[2]) * input_vector[row[1]]
            elif row[3] == '+':
                MEM[row[2]] = MEM[row[0]] + MEM[row[1]]
        else:
            output_vector[row[0]] = MEM[row[1]]
    
    return output_vector

## Test

In [28]:
#num = 1000
bit = 4
INT_range = 2**bit - 1
height = 16
weight = 16

In [29]:
np.random.seed(100)
matrix = np.random.randint(low=-INT_range, high=INT_range,size=(height, weight))

In [30]:
index_array = preprocess_matrix_to_array(matrix)

Size： 256
Shape： (16, 16)
Dimension 2
[[ -7   9 -12  -8   8   0   1  -5   5 -13   6 -13 -13  -1 -13   2]
 [  1   9   0 -11  -4  13  11   1  12  -6  14   7 -13  12  -3 -11]
 [-14  -2   6   4 -11 -11  12  12 -12  -8   2   0 -14  14  -1   8]
 [ -8  12   1 -13  10  -6   4 -13   6  12   9  -1   2   1   0  -8]
 [ -2  -9  -3   3 -15   9 -13   6  -5   2  -7  -2  -5   2   6 -11]
 [  3  -7  12  12   4   6  -1 -15  -2  10  -3  -5 -12  -9 -12   5]
 [  0  -5   8   7   0  13 -12  -6   1   5  -4 -11 -10  -8  -9   7]
 [-13  -5   5   3  -2   8  -3 -14  11  -9  -5   7 -15  11   8 -13]
 [  4   6 -11   3 -11 -12   8  14  -6   1   1   6  10   7  -9 -10]
 [ -9  11   5  11  -8  -4   4  10 -13   4   9   8   2  -2 -10  10]
 [ -2 -12   1  -3   7   3  -2  -3 -12  13   5  -5  -7  10  -7  -3]
 [  6   3  14   8  13  -3   6  10 -15  -6  -7   7  13 -13   1 -10]
 [  4  11 -13 -12  -1   7  -2   5 -14  -1   4 -14   5  -7   9  14]
 [ 11  13   3   3  -8 -13   2 -13   8   2  -2  11  11  14 -15  -5]
 [  6 -12  13 -10   3  -

MEM[211] = MEM[59] + MEM[84];
MEM[212] = MEM[69] + MEM[81];
MEM[213] = MEM[70] + MEM[87];
MEM[214] = MEM[71] + MEM[81];
MEM[215] = MEM[73] + MEM[78];
MEM[216] = MEM[73] + MEM[88];
MEM[217] = MEM[77] + MEM[79];
MEM[218] = MEM[77] + MEM[122];
MEM[219] = MEM[79] + MEM[92];
MEM[220] = MEM[80] + MEM[90];
MEM[221] = MEM[80] + MEM[113];
MEM[222] = MEM[82] + MEM[88];
MEM[223] = MEM[82] + MEM[119];
MEM[224] = MEM[90] + MEM[103];
MEM[225] = MEM[92] + MEM[111];
MEM[226] = MEM[95] + MEM[97];
MEM[227] = MEM[97] + MEM[110];
MEM[228] = MEM[98] + MEM[99];
MEM[229] = MEM[98] + MEM[132];
MEM[230] = MEM[101] + MEM[104];
MEM[231] = MEM[101] + MEM[109];
MEM[232] = MEM[102] + MEM[108];
MEM[233] = MEM[103] + MEM[125];
MEM[234] = MEM[104] + MEM[106];
MEM[235] = MEM[108] + MEM[110];
MEM[236] = MEM[112] + MEM[127];
MEM[237] = MEM[113] + MEM[142];
MEM[238] = MEM[117] + MEM[124];
MEM[239] = MEM[117] + MEM[136];
MEM[240] = MEM[120] + MEM[126];
MEM[241] = MEM[120] + MEM[130];
MEM[242] = MEM[121] + MEM[135];
MEM[243

In [31]:
np.random.seed(10)
input_vector = np.random.randint(low=-INT_range,high=INT_range,size=(weight,1))

In [32]:
matrix_optimized_multiplier(index_array, input_vector)

array([[-139],
       [ 692],
       [ 288],
       [ 208],
       [  63],
       [-525],
       [-519],
       [ -29],
       [ 859],
       [ 482],
       [-169],
       [-354],
       [-217],
       [ 603],
       [-368],
       [ 356]])

In [33]:
np.dot(matrix, input_vector)

array([[-139],
       [ 692],
       [ 288],
       [ 208],
       [  63],
       [-525],
       [-519],
       [ -29],
       [ 859],
       [ 482],
       [-169],
       [-354],
       [-217],
       [ 603],
       [-368],
       [ 356]])

In [34]:
index_array2, bin_matrix = minimize_operation_matrix(matrix, bit_size = 5)


 Max number of 1: 13

 Max number of 1: 12

 Max number of 1: 11

 Max number of 1: 10

 Max number of 1: 9

 Max number of 1: 8

 Max number of 1: 7

 Max number of 1: 6

 Max number of 1: 5

 Max number of 1: 4

 Max number of 1: 3

 Max number of 1: 2

 Max number of 1: 1
Increasing matrix size, current bin_max_length: 240 bin_weight: 80 cnt: 158
Increasing matrix size, updated bin_max_length: 480


In [35]:
index_array2

[[16, 16, 5, 'parameters'],
 [0, 0, 4, '-<<'],
 [1, 0, 3, '<<'],
 [2, 0, 2, '<<'],
 [3, 0, 1, '<<'],
 [4, 0, 0, '<<'],
 [5, 1, 4, '-<<'],
 [6, 1, 3, '<<'],
 [7, 1, 2, '<<'],
 [8, 1, 1, '<<'],
 [9, 1, 0, '<<'],
 [10, 2, 4, '-<<'],
 [11, 2, 3, '<<'],
 [12, 2, 2, '<<'],
 [13, 2, 1, '<<'],
 [14, 2, 0, '<<'],
 [15, 3, 4, '-<<'],
 [16, 3, 3, '<<'],
 [17, 3, 2, '<<'],
 [18, 3, 1, '<<'],
 [19, 3, 0, '<<'],
 [20, 4, 4, '-<<'],
 [21, 4, 3, '<<'],
 [22, 4, 2, '<<'],
 [23, 4, 1, '<<'],
 [24, 4, 0, '<<'],
 [25, 5, 4, '-<<'],
 [26, 5, 3, '<<'],
 [27, 5, 2, '<<'],
 [28, 5, 1, '<<'],
 [29, 5, 0, '<<'],
 [30, 6, 4, '-<<'],
 [31, 6, 3, '<<'],
 [32, 6, 2, '<<'],
 [33, 6, 1, '<<'],
 [34, 6, 0, '<<'],
 [35, 7, 4, '-<<'],
 [36, 7, 3, '<<'],
 [37, 7, 2, '<<'],
 [38, 7, 1, '<<'],
 [39, 7, 0, '<<'],
 [40, 8, 4, '-<<'],
 [41, 8, 3, '<<'],
 [42, 8, 2, '<<'],
 [43, 8, 1, '<<'],
 [44, 8, 0, '<<'],
 [45, 9, 4, '-<<'],
 [46, 9, 3, '<<'],
 [47, 9, 2, '<<'],
 [48, 9, 1, '<<'],
 [49, 9, 0, '<<'],
 [50, 10, 4, '-<<'],
 

In [36]:
matrix_optimized_multiplier(index_array2, input_vector)

array([[-139],
       [ 692],
       [ 288],
       [ 208],
       [  63],
       [-525],
       [-519],
       [ -29],
       [ 859],
       [ 482],
       [-169],
       [-354],
       [-217],
       [ 603],
       [-368],
       [ 356]])